Spatial SEIRS Model Simulation with Automated Network Information Processing 
by Xiyuan Li

Western University
London, Ontario
July 21st, 2020

Virsion beta 1.0.1 Under Testing


In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [ ]:
class objectClass(object):
    def __init__(self, number):
        self.number = number

tau = np.identity(5) 
row = len(tau)
tau=np.rot90(np.fliplr(tau))

x=[]
y=[]
num = 11*row

for i in range(11*row): #elements and first derivatives
    x.append(objectClass(i).number)
for i in range(row*row): #sudo_matrix
    y.append(objectClass(i).number)

In [ ]:
j = 0
sudo_matrix=[]
while j < row:
    i = 0 #0
    while i < row:
        sudo_matrix.append(tau[j,i])
        i+=1 #1
    j+=1 #1
y = sudo_matrix

In [ ]:
#Initial Value Assignement
beta = 1.12
roe = 0.5
kappa = 0.27
gamma = 0.29
alpha=0.1
delta = 0.04
days = 200
init = (292970, 10, 10, 10, 0, 29297, 1, 1, 1, 0, 29297, 1, 1, 1, 0,500000, 1, 1, 1, 0,29297, 1, 1, 1, 0)
initial = (292970, 10, 10, 10, 0, 29297, 1, 1, 1, 0, 29297, 1, 1, 1, 0,500000, 1, 1, 1, 0,29297, 1, 1, 1, 0, 293000,29300,29300,500003,29300)
params = beta, kappa, roe, gamma, delta
ODEmodelInitial = init

In [ ]:
def sum_organizer(sudo_matrix):
    
    g = []
    for i in range(row*3):
        g.append(objectClass(i).number)
    m_count = 0
    for n in range(row):
        count = row #3
        while count >0:
            count-=1 #2
            g[m_count*3+0] = g[m_count*3+0] + x[5*count+2]*y[count+m_count*row]
            g[m_count*3+1] = g[m_count*3+1] + x[5*count+3]*y[count+m_count*row]
            g[m_count*3+2] = g[m_count*3+2] + x[5*row+count]*y[count+m_count*row]
        m_count+=1
        
    return g

In [ ]:
def get_tau_self(count):

    f = []
    for i in range(row):
        f.append(y[int(count)+i*row])
        
    return f

In [ ]:
def ode_model(z, t, beta, kappa, roe, gamma, delta):
    
    number = 0
    while number < 5*row:
        x[number] = float(z[number])
        number+=1
    count = 0
    iteration = 0
    while count < num:
        f=get_tau_self(iteration)
        g=sum_organizer(y)
        #reference: N1 = S1 + E1 + Ic1 + Isc1 + R1
        x[iteration+row*5] = x[iteration*5] + x[iteration*5+1] + x[iteration*5+2] + x[iteration*5+3] + x[iteration*5+4]
        n2 = 0
        sum = 0
        while n2 < row:
            sum = sum + f[n2]*(g[n2*3]+alpha*g[n2*3+1])/g[n2*3+2]
            n2+=1
        x[row*6-1+iteration*5+1] = -beta*x[iteration*5]*((sum)) + delta*x[iteration*5+4]
        x[row*6-1+iteration*5+2] = beta*x[iteration*5]*((sum))- kappa*x[iteration*5+1]
        x[row*6-1+iteration*5+3] = roe*kappa*x[iteration*5+1] - gamma*x[iteration*5+2]
        x[row*6-1+iteration*5+4] = (1-roe)*kappa*x[iteration*5+1] - gamma*x[iteration*5+3]
        x[row*6-1+iteration*5+5] = gamma*(x[iteration*5+2] + x[iteration*5+3]) - delta*x[iteration*5+4]
        count+=11
        iteration+=1
        
    number2 = 0
    output = ()
    while number2 < row:
        i = 0
        while i < 5:
            i+=1
            output = output + (x[row*6-1+5*number2+i],)
        number2+=1
   
    return output

In [ ]:
def ode_solver(t, ODEmodelInitial, params):
    res = odeint(ode_model, ODEmodelInitial, t, args=(params))
    return res

In [ ]:
def main(ODEmodelInitial, params, days):
    
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, ODEmodelInitial, params)
    number = 0
    while number < 5*row:
        x[number] = sol[:,number]
        number+=1   

    graphics=0
    s=0
    e=0
    ic=0
    isc=0
    r=0
    while graphics < row:
    
        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x=tspan, y=x[5*graphics+0], mode='lines+markers', name='Susceptible ' + str(graphics+1)))
        fig1.add_trace(go.Scatter(x=tspan, y=x[5*graphics+1], mode='lines+markers', name='Exposed '+ str(graphics+1)))
        fig1.add_trace(go.Scatter(x=tspan, y=x[5*graphics+2], mode='lines+markers', name='Clinical '+ str(graphics+1)))
        fig1.add_trace(go.Scatter(x=tspan, y=x[5*graphics+3], mode='lines+markers', name='Subclinical '+ str(graphics+1)))
        fig1.add_trace(go.Scatter(x=tspan, y=x[5*graphics+4], mode='lines+markers',name='Removed '+ str(graphics+1)))
        s=s+x[5*graphics+0]
        e=e+x[5*graphics+1]
        ic=ic+x[5*graphics+2]
        isc=isc+x[5*graphics+3]
        r=r+x[5*graphics+4]
        
        if days <= 30:
            step = 1
        elif days <= 90:
            step = 7
        else:
            step = 30
            
        fig1.update_layout(title='SEIIRS Community ' + str(graphics+1),
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600,
                     )
    
        fig1.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    
    
        
        fig1.show()
        graphics+=1
        
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=tspan, y=s, mode='lines+markers', name='Susceptible ' ))
    fig2.add_trace(go.Scatter(x=tspan, y=e, mode='lines+markers', name='Exposed '))
    fig2.add_trace(go.Scatter(x=tspan, y=ic, mode='lines+markers', name='Clinical '))
    fig2.add_trace(go.Scatter(x=tspan, y=isc, mode='lines+markers', name='Subclinical '))
    fig2.add_trace(go.Scatter(x=tspan, y=r, mode='lines+markers',name='Removed '))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
        
    fig2.update_layout(title='SEIIRS Community Combined',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600,
                     )
    
    fig2.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig2.show()

In [ ]:
main(ODEmodelInitial, params, days)